In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
def pop_rec(data, n=5):
    pop = data.groupby(['item_id'])['sales_value'].sum().reset_index()
    pop.sort_values(['sales_value'], ascending=False, inplace=True)
    
    return pop.head(n).item_id.tolist()

In [6]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    # Группируем по item_id и суммируем sales_value
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    # Отсекаем товары с продажами ниже среднего значения
    popular.loc[popular['sales_value'] > popular['sales_value'].mean()]
    
    # Добавляем колонку sales_normalized и нормализуем значения
    popular['sales_normalized'] = (popular['sales_value'] - popular['sales_value'].min()) \
                                        / (popular['sales_value'].max() - popular['sales_value'].min())
    
    # Возвращаем топ n товаров
    recs = popular.head(n)
    
    return recs

In [7]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные рекомендации"""
        
    # Веса - логарифмы колонки sales_normalized, полученные от функции popularity_recommendation
    items_weights['weight_log'] = np.log(items_weights['sales_normalized'])
    items_weights.sort_values('weight_log', ascending=False, inplace=True)
    
    # Случайная выборка из товаров с наибольшими весами
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False)
    
    return recs

In [8]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    # Вызываем функцию poplarity_recommendation для препобразования начальной выборки
    pop_items = popularity_recommendation(items, n)
    
    # Результат popularity_recommendation передаем в функцию weighted_random_recommendation для логарифмирования
    # и случайной выборки 
    recs = weighted_random_recommendation(pop_items, n)
    
    return recs.tolist()

In [9]:
%%time

random_recommendation(data_train, n=6)

CPU times: user 67.7 ms, sys: 7.85 ms, total: 75.5 ms
Wall time: 70.4 ms


[916122, 1082185, 6534166, 6534178, 1029743, 6533889]

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [10]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [11]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [12]:
import warnings
warnings.filterwarnings('ignore')
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [13]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [16]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

In [17]:
result.columns=['user_id', 'actual']

In [18]:
%%time

result['popular_recommendation'] = result['user_id'].apply(lambda x: pop_rec(data_train.loc[data_train['item_id'].isin(top_5000),:], n=5))
result.head(2)

CPU times: user 8min 12s, sys: 14.6 s, total: 8min 26s
Wall time: 8min 26s


In [19]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

CPU times: user 49.2 ms, sys: 0 ns, total: 49.2 ms
Wall time: 48.3 ms


In [20]:
items = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items.sort_values('sales_value', ascending=False, inplace=True)

In [21]:
items

,item_id,sales_value
0,6666,2.997551e+06
4346,6534178,4.477999e+05
4339,6533889,4.048334e+04
2757,1029743,3.576466e+04
4345,6534166,3.017077e+04
...,...,...
4979,14050436,1.769000e+01
4978,14043826,1.665000e+01
4975,14043817,1.604000e+01
4981,14050461,1.476000e+01


In [22]:
%%time

items = items.head(100)

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=9))

CPU times: user 10.4 s, sys: 8.25 ms, total: 10.5 s
Wall time: 10.4 s


In [23]:
from metrics import precision_at_k, recall_at_k

In [24]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.1552:popular_recommendation
0.192:itemitem
0.1104:random_recommendation
